In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
train = pd.read_csv('C:/222/large_data/kaggle/mnist/train.csv')
test = pd.read_csv('C:/222/large_data/kaggle/mnist/test.csv')
y = train['label']

In [3]:
train = train.drop(columns=['label'])

In [4]:
batch_size = 16
classes = y.unique()
num_classes = np.shape(classes)[0]

In [5]:
train_data = np.array(train).reshape(-1, 28, 28)
test_data = np.array(test).reshape(-1, 28, 28)

In [6]:
X_train = train_data.reshape(-1, 28, 28, 1)
X_test = test_data.reshape(-1, 28, 28, 1)

In [7]:
training_iters = 10 
learning_rate = 1e-3 
batch_size = X_train.shape[0]/16

In [8]:
#both placeholders are of type float
x = tf.placeholder("float", shape = [None, X_train.shape[1], X_train.shape[2], X_train.shape[3]], name = 'x')
y = tf.placeholder("float", shape = [None, num_classes], name = 'labels')

In [9]:
def conv2d(x, W_kernel, bias, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W_kernel, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, bias)
    return tf.nn.relu(x) 

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [10]:
weights = {
    'wc1': tf.get_variable('W0', shape=(3,3,1,32), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc2': tf.get_variable('W1', shape=(3,3,32,64), initializer=tf.contrib.layers.xavier_initializer()), 
    'wc3': tf.get_variable('W2', shape=(3,3,64,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'wd1': tf.get_variable('W3', shape=(4*4*128,128), initializer=tf.contrib.layers.xavier_initializer()), 
    'out': tf.get_variable('W6', shape=(128, num_classes), initializer=tf.contrib.layers.xavier_initializer()), 
}
biases = {
    'bc1': tf.get_variable('B0', shape=(32), initializer=tf.contrib.layers.xavier_initializer()),
    'bc2': tf.get_variable('B1', shape=(64), initializer=tf.contrib.layers.xavier_initializer()),
    'bc3': tf.get_variable('B2', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'bd1': tf.get_variable('B3', shape=(128), initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable('B4', shape=(10), initializer=tf.contrib.layers.xavier_initializer()),
}


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


In [24]:
def conv_net(x, weights, biases):  

    # here we call the conv2d function we had defined above and pass the input image x, weights wc1 and bias bc1.
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 14*14 matrix.
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    # here we call the conv2d function we had defined above and pass the input image x, weights wc2 and bias bc2.
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 7*7 matrix.
    conv2 = maxpool2d(conv2, k=2)

    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling), this chooses the max value from a 2*2 matrix window and outputs a 4*4.
    conv3 = maxpool2d(conv3, k=2)


    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Output, class prediction
    # finally we multiply the fully connected layer with the weights and add a bias term. 
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [25]:
pred = conv_net(x, weights, biases)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [26]:
#Here you check whether the index of the maximum value of the predicted image is equal to the actual labelled image. and both will be a column vector.
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))

#calculate accuracy across all the given images and average them out. 
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [27]:
init = tf.global_variables_initializer()

In [34]:
len(X_train)//batch_size

16.0

In [36]:
with tf.Session() as sess:
    sess.run(init) 
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    for i in range(training_iters):
        for batch in range(int(len(X_train)/batch_size)):
            batch_x = X_train[batch*batch_size:min((batch+1)*batch_size,len(X_train))]
            batch_y = y[batch*batch_size:min((batch+1)*batch_size,len(y))]    
            # Run optimization op (backprop).
                # Calculate batch loss and accuracy
            opt = sess.run(optimizer, feed_dict={x: batch_x,
                                                              y: batch_y})
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y})
        print("Iter " + str(i) + ", Loss= " + \
                      "{:.6f}".format(loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(acc))
        print("Optimization Finished!")

        # Calculate accuracy for all 10000 mnist test images
        test_acc,valid_loss = sess.run([accuracy,cost], feed_dict={x: test_X,y : test_y})
        train_loss.append(loss)
        test_loss.append(valid_loss)
        train_accuracy.append(acc)
        test_accuracy.append(test_acc)
        print("Testing Accuracy:","{:.5f}".format(test_acc))
    summary_writer.close()

TypeError: slice indices must be integers or None or have an __index__ method

In [12]:
# def create_convolutional_layer(input,
#                num_input_channels, 
#                conv_filter_size,        
#                num_filters):  
    
#     ## We shall define the weights that will be trained using create_weights function.
#     weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
#     ## We create biases using the create_biases function. These are also trained.
#     biases = create_biases(num_filters)

#     ## Creating the convolutional layer
#     layer = tf.nn.conv2d(input=input,
#                      filter=weights,
#                      strides=[1, 1, 1, 1],
#                      padding='SAME')

#     layer += biases

#     ## We shall be using max-pooling.  
#     layer = tf.nn.max_pool(value=layer,
#                             ksize=[1, 2, 2, 1],
#                             strides=[1, 2, 2, 1],
#                             padding='SAME')
#     ## Output of pooling is fed to Relu which is the activation function for us.
#     layer = tf.nn.relu(layer)
#     return layer

In [13]:
# def create_flatten_layer(layer):
#     layer_shape = layer.get_shape()
#     num_features = layer_shape[1:4].num_elements()
#     layer = tf.reshape(layer, [-1, num_features])
#     return layer

In [14]:
# layer_shape = layer.get_shape()
# num_features = layer_shape[1:4].num_elements()

In [15]:
# def create_fc_layer(input,          
#              num_inputs,    
#              num_outputs,
#              use_relu=True):
#     """
#     creating fully-connected layer 
#     """
    
#     #Let's define trainable weights and biases.
#     weights = create_weights(shape=[num_inputs, num_outputs])
#     biases = create_biases(num_outputs)
 
#     layer = tf.matmul(input, weights) + biases
#     if use_relu:
#         layer = tf.nn.relu(layer)
 
#     return layer

In [16]:
# x = tf.placeholder(tf.float32, shape=[None, 28, 28 , 1], name='x')
# y_true = tf.placeholder(tf.float32, shape=[None, len(y)], name='y_true')
# # y_true_cls = tf.argmax(y_true, dimension=10)

In [17]:
# layer_c1 = create_convolutional_layer(input = x,
#                num_input_channels = 1, 
#                conv_filter_size = (3),        
#                num_filters = 1)
# layer_fl = create_flatten_layer(layer_c1)
# layer_out = create_fc_layer(layer_fl,          
#              num_inputs = (layer_fl.get_shape()[1:].num_elements()),    
#              num_outputs = num_classes,
#              use_relu = True)

In [18]:
# y_pred = tf.nn.softmax(layer_out, name = "y_pred")
# y_pred_cls = tf.argmax(y_pred, dimension = 10)

In [19]:
# cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=y_pred,
#                                                     labels=y_true)
# cost = tf.reduce_mean(cross_entropy)

In [20]:
# sess = tf.Session()

In [21]:
# optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [22]:
# # batch_size = 16

# feed_dict_train = {x: X_train,
#                            y_true: y}
 
# sess.run(optimizer, feed_dict=feed_dict_train)